This notebook runs fullkv on GSM 8K

## Setup

In [1]:
import os
import time
import json
import asyncio
from tqdm import tqdm
from dataclasses import dataclass

import sglang as sgl

os.makedirs("./z_experiment/evaluation/", exist_ok=True)
os.makedirs("./z_experiment/output/", exist_ok=True)
os.makedirs("./z_experiment/results/", exist_ok=True)


@dataclass
class Config:
    model_path: str = "deepseek-ai/DeepSeek-R1-Distill-Llama-8B"
    save_path: str = "./z_experiment/output/test_output.jsonl"
    data_size: int = 10


config = Config()

In [2]:
# load data
PROMPT_TEMP = "You are given a math problem.\n\nProblem: {question}\n\n You need to solve the problem step by step. First, you need to provide the chain-of-thought, then provide the final answer.\n\n Provide the final answer in the format: Final answer:  \\boxed{{}}"

test_data = []
prompts = []
with open("./evaluation/data/test_one.jsonl") as f:
    for idx, line in enumerate(f):
        if idx == config.data_size:
            break
        sample = json.loads(line)
        prompt = PROMPT_TEMP.format(question=sample["question"])
        prompts.append(prompt)

        sample["prompt"] = prompt
        sample["index"] = idx

        test_data.append(sample)

## load model

In [3]:
compress_algorithm = "CustomKV" # "SnapKV", "CustomKV"
compress_max_window = 8
compress_max_prompt = 128
compress_divide_length = 64
compress_divide_method = "newline" # "step_length", "newline"

llm = sgl.Engine(
    model_path=config.model_path,
    dtype="bfloat16",
    disable_overlap_schedule=True,
    compress_algorithm=compress_algorithm,
    compress_max_window=compress_max_window,
    compress_max_prompt=compress_max_prompt,
    compress_divide_length=compress_divide_length,
    compress_divide_method=compress_divide_method,
)


def main():
    sampling_params = {"temperature": 0.0, "top_p": 0.95, "max_new_tokens": 8192}

    start_time = time.time()
    outputs = llm.generate(prompts, sampling_params)
    end_time = time.time()

    llm.shutdown()

    for sample_idx, output in enumerate(outputs):
        test_data[sample_idx]["output"] = output["text"]
        test_data[sample_idx]["prefill_tokens"] = output["meta_info"]["prompt_tokens"]
        test_data[sample_idx]["output_tokens"] = output["meta_info"][
            "completion_tokens"
        ]
        test_data[sample_idx]["total_tokens"] = (
            output["meta_info"]["prompt_tokens"]
            + output["meta_info"]["completion_tokens"]
        )

    with open(config.save_path, "w") as fp:
        for line in test_data:
            fp.write(json.dumps(line) + "\n")

    total_time = end_time - start_time
    total_tokens_generated = sum(
        output["meta_info"]["completion_tokens"] for output in outputs
    )
    throughput_tokens = total_tokens_generated / total_time
    throughput_requests = len(prompts) / total_time

    print(f"Total execution time: {total_time:.2f} seconds")
    print(f"Throughput (tokens/s): {throughput_tokens:.2f}")
    print(f"Throughput (requests/s): {throughput_requests:.2f}")

/tmp/kewan/.conda/envs/sglang-dev/lib/python3.12/site-packages/tqdm/auto.py:21: TqdmWarning: IProgress not found. Please update jupyter and ipywidgets. See https://ipywidgets.readthedocs.io/en/stable/user_install.html
  from .autonotebook import tqdm as notebook_tqdm
/tmp/kewan/.conda/envs/sglang-dev/lib/python3.12/site-packages/transformers/models/auto/image_processing_auto.py:590: FutureWarning: The image_processor_class argument is deprecated and will be removed in v4.42. Please use `slow_image_processor_class`, or `fast_image_processor_class` instead
  warnings.warn(


INFO 04-03 03:30:51 __init__.py:190] Automatically detected platform cuda.


2025-04-03 03:30:51,523	INFO util.py:154 -- Missing packages: ['ipywidgets']. Run `pip install -U ipywidgets`, then restart the notebook server for rich notebook output.
2025-04-03 03:30:53,191 - INFO - flashinfer.jit: Prebuilt kernels not found, using JIT backend
/tmp/kewan/.conda/envs/sglang-dev/lib/python3.12/site-packages/transformers/models/auto/image_processing_auto.py:590: FutureWarning: The image_processor_class argument is deprecated and will be removed in v4.42. Please use `slow_image_processor_class`, or `fast_image_processor_class` instead
  warnings.warn(
/tmp/kewan/.conda/envs/sglang-dev/lib/python3.12/site-packages/transformers/models/auto/image_processing_auto.py:590: FutureWarning: The image_processor_class argument is deprecated and will be removed in v4.42. Please use `slow_image_processor_class`, or `fast_image_processor_class` instead
  warnings.warn(


INFO 04-03 03:30:57 __init__.py:190] Automatically detected platform cuda.
INFO 04-03 03:30:57 __init__.py:190] Automatically detected platform cuda.


2025-04-03 03:30:59,605 - INFO - flashinfer.jit: Prebuilt kernels not found, using JIT backend
Loading safetensors checkpoint shards:   0% Completed | 0/2 [00:00<?, ?it/s]
Loading safetensors checkpoint shards:  50% Completed | 1/2 [00:01<00:01,  1.11s/it]
Loading safetensors checkpoint shards: 100% Completed | 2/2 [00:02<00:00,  1.09s/it]
Loading safetensors checkpoint shards: 100% Completed | 2/2 [00:02<00:00,  1.09s/it]



In [4]:
prompts = prompts[:100]

if __name__ == "__main__":
    main()

2025-04-03 03:31:08,609 - INFO - flashinfer.jit: Loading JIT ops: batch_prefill_with_kv_cache_dtype_q_bf16_dtype_kv_bf16_dtype_o_bf16_dtype_idx_i32_head_dim_qk_128_head_dim_vo_128_posenc_0_use_swa_False_use_logits_cap_False_f16qk_False
2025-04-03 03:31:08,642 - INFO - flashinfer.jit: Finished loading JIT ops: batch_prefill_with_kv_cache_dtype_q_bf16_dtype_kv_bf16_dtype_o_bf16_dtype_idx_i32_head_dim_qk_128_head_dim_vo_128_posenc_0_use_swa_False_use_logits_cap_False_f16qk_False
2025-04-03 03:31:09,548 - INFO - flashinfer.jit: Loading JIT ops: batch_decode_with_kv_cache_dtype_q_bf16_dtype_kv_bf16_dtype_o_bf16_dtype_idx_i32_head_dim_qk_128_head_dim_vo_128_posenc_0_use_swa_False_use_logits_cap_False
2025-04-03 03:31:09,581 - INFO - flashinfer.jit: Finished loading JIT ops: batch_decode_with_kv_cache_dtype_q_bf16_dtype_kv_bf16_dtype_o_bf16_dtype_idx_i32_head_dim_qk_128_head_dim_vo_128_posenc_0_use_swa_False_use_logits_cap_False


KeyboardInterrupt: 

## Evaluation

Use the following commands to evaluate GSM8k results.

```python
python evaluation/math_eval_all_v2.py \
    --exp_name "evaluation" \
    --output_dir "./z_experiment/results" \
    --base_dir "./z_experiment/output" \
    --dataset gsm8k
```